In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! unzip  /content/drive/MyDrive/nycu2023mlfinalproject.zip -d ./data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


# Loading the dataset (using the full dataset for training)
dataset = ImageFolder(root='/content/data/data/train', transform=data_transforms)

# DataLoader
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

# Model setup (using ResNet152)
model = models.resnet152(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, len(dataset.classes))
)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5.5e-5)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

# Transfer to GPU if available
model.to(device)

# Training loop
best_accuracy = 0.0
for epoch in range(20):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)

    scheduler.step()
    epoch_acc = running_corrects.double() / len(dataset)
    if epoch_acc > best_accuracy:
        best_accuracy = epoch_acc
        torch.save(model.state_dict(), '/content/drive/MyDrive/best.pth')

    print(f'Epoch {epoch+1}, Training Accuracy: {epoch_acc:.4f}')

# Save the final model
torch.save(model.state_dict(), '/content/drive/MyDrive/final.pth')
